# MUTAG Classification by using Diff-Pool

### Yonsei App. Stat.
### Sunwoo Kim

### Source data : https://pubs.acs.org/doi/abs/10.1021/jm00106a046
### Diff-pool : https://arxiv.org/abs/1806.08804

MUTAG 데이터는 각 분자(graph)가 mutagenicity를 갖고 있는지 분류하는 task입니다.  
각 node feature는 7개의 화학 원자 중 하나이며,  
각 edge feature는 연결 종류를 의미합니다.  
여기서는 Ying et al. 19.에서 제시한 hierarchical clustering인 diff-pool로 task를 수행해보겠습니다.

### 1. Importing required packages

In [45]:
from GNN_models import *
import torch
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset = TUDataset(root='/tmp/MUTAG', name='MUTAG', use_node_attr=True)
train_data = dataset[:150]
test_data = dataset[150:]

In [19]:
loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [12]:
data_ = dataset[0]

In [26]:
mean_node = 0
proportion_y
total_y = []
for i in range(188) : 
    data = dataset[i]
    mean_node += data.x.shape[0]
    total_y.append(data.y.item())

print()
print(mean_node/188)
print(set(total_y))

17.930851063829788
{0, 1}


각 분자는 평균적으로 17개의 원자를 갖고있고, 최종 class는 0 또는 1입니다.  
이 정보를 토대로 아래와 같이 모델을 정의합니다.

### 2. Define model

In [28]:
model = diffpool_gnn(dataset = data_,  # Tell model the dimension of input and output
                    latent_dim = [16,16, "d",16,16], # Dimension of hidden SAGE layers
                    diff_dim = [16, 2],  # Dimension of diff-pool layer and dimensionality reduction
                    end_dim = [16, 2], # Dimension of output dimension
                    device = device) # Readout layer's dimension

모델은 아래와 같은 구조를 갖고 있습니다.  
GraphSAGE -> GraphSAGE -> DiffPool -> GraphSAGE -> GraphSAGE -> DiffPool -> Readout  
첫 Diffpool은 원래 노드 수에서 2개로 각 노드에 대해 차원축소를 진행합니다.  
두번째 diffpool은 하나의 군집으로 두 군집을 축소합니다.

### 3. Train model & Accuracy check

학습을 시작해보겠습니다.

In [53]:
model.reset_parameters()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
cost = torch.nn.CrossEntropyLoss()

i = 0
model.to(device)
model.train()
for epoch in range(200):
    epoch_loss = torch.zeros(1).to(device)
    for data in loader : 
        data.to(device)
        optimizer.zero_grad()
        out = model(data = data)
        loss = cost(out, data.y)
        loss += 0.1*model.frobenious_norm() # L_lp loss
        loss += 0.1*model.cross_entropy() # L_E loss
        loss.backward()
        optimizer.step()
        epoch_loss += loss
    if i%10 == 0 : 
        print("Epoch : {0} / Loss : {1}".format(i, epoch_loss.to("cpu").detach().item()/150))
    i += 1

Epoch : 0 / Loss : 6.21368896484375
Epoch : 10 / Loss : 5.146040445963542
Epoch : 20 / Loss : 5.035551350911458
Epoch : 30 / Loss : 4.920035400390625
Epoch : 40 / Loss : 4.837781575520833
Epoch : 50 / Loss : 4.799435628255209
Epoch : 60 / Loss : 4.769701334635417
Epoch : 70 / Loss : 4.7520658365885415
Epoch : 80 / Loss : 4.727667643229167
Epoch : 90 / Loss : 4.775590006510416
Epoch : 100 / Loss : 4.724498697916666
Epoch : 110 / Loss : 4.695520833333333
Epoch : 120 / Loss : 4.722896321614583
Epoch : 130 / Loss : 4.669930013020833
Epoch : 140 / Loss : 4.677373453776042
Epoch : 150 / Loss : 4.638875325520833
Epoch : 160 / Loss : 4.696876220703125
Epoch : 170 / Loss : 4.642325846354167
Epoch : 180 / Loss : 4.6255415852864585
Epoch : 190 / Loss : 4.643114827473958


Test data에 대해서 성능평가를 진행해보겠습니다.

In [54]:
acc = 0
for test_d in test_data : 
    if  torch.argmax(model(test_d.to(device))) == test_d.y : 
        acc += 1
        
print("Test accuracy {0}".format(acc/38))

Test accuracy 0.9210526315789473


92%라는 꽤 높은 정확도를 보이는 것을 확인할 수 있습니다.